<a href="https://colab.research.google.com/github/brandonso994/AttnLSTMMusicGeneration/blob/main/V5/Generate_Notes_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_self_attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=ad380bb2f2fffe95ab7b152d255f6ae252d1dfff08d5c8db7fab4628235fdb4d
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [ ]:
from music21 import converter, instrument, note, chord, stream, volume
from fractions import Fraction
import matplotlib.pyplot as plt
import glob
import numpy as np
import nltk
import pandas as pd
import pickle
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Bidirectional, BatchNormalization, Input, Embedding, Concatenate
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import CategoricalAccuracy
import tensorflow as tf
from keras import backend as K
from keras_self_attention import SeqSelfAttention

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def flatten(array):
  new_array = [item for array in array for item in array]
  return new_array

# Create Sequence with window length = sequence_length for input into model
def sequence(notes ,all_notes, note_set_len, sequence_length, midi_input):

  note_values = sorted(set(note for note in all_notes))

  note_encode = dict((note, num) for num, note in enumerate(note_values))

  network_input_note = []
  note_output = []

  for i in range (len(notes) - sequence_length):
    note_sequence = notes[i:i + sequence_length]
    note_input_sequence = [note_encode[note] for note in note_sequence]

    network_input_note.append(note_input_sequence)
    note_output.append(note_encode[notes[i + sequence_length]])

    if midi_input:
      break

  n_patterns = len(network_input_note)

  network_input_note = np.array(network_input_note)
  network_input_note = np.reshape(network_input_note, (network_input_note.shape[0], network_input_note.shape[1], 1))

  note_output = np.array(note_output)

  note_output = np.reshape(note_output, (note_output.shape[0], 1))


  return network_input_note

def create_model(note_set_len, version_num):

    model = Sequential()

    # Note input
    model.add(Input(shape=(None,)))
    model.add(Embedding(note_set_len, 600))
    model.add(Bidirectional(LSTM(1024, return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(LSTM(1024, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(1024, return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(note_set_len, activation='softmax', name='note'))

    model.load_weights('/content/drive/My Drive/MRP/Model/model_weights_' + version_num + '_checkpoints.h5')

    return model

# Predicts the next note given the sequence
def note_prediction(model, note_input, notes, seq_len, num_notes=500, temp=1, midi_input=False):
    note_set_len = len(set(notes))

    note_values = sorted(set(note for note in notes))

    note_decode = dict((num, note) for num, note in enumerate(note_values))
    note_encode = dict((note, num) for num, note in enumerate(note_values))

    # Whether input was given, or start randomly from sequence
    if midi_input:
      start = 0
    else:
      start = np.random.randint(0, len(note_input) - 1)

    generated_notes = []
    note_pattern = note_input[start].copy()

    #print(note_pattern)

    # Loop, predicting each note and appending the prediction to the sequence, removing the oldest note to preserve sequence length
    for item in range(num_notes):
        note_pattern_reshaped = np.reshape(note_pattern, (1, seq_len, 1))

        predictions = model.predict(note_pattern_reshaped, verbose=0)

        predicted_note = predictions[0]

        # Apply temperature
        predicted_note = np.log(predicted_note) / temp

        note_probs = np.exp(predicted_note) / np.sum(np.exp(predicted_note))

        # Sample the next pitch and duration using the temperature-adjusted probabilities
        next_note = np.random.choice(len(note_probs), p=note_probs)

        decoded_note = note_decode[next_note]
        generated_notes.append(decoded_note)
        next_note_encode = note_encode[decoded_note]
        note_pattern = np.append(note_pattern[1:], [next_note_encode])

    return generated_notes

# Generate midi from predicted notes
def generate_midi(note_sequence, file_num, version_num):
  output_notes = []
  offset = 0
  isRest = False
  for output_note in note_sequence:
      output_note = output_note.split()
      temp = output_note[0]
      duration = float(output_note[1].split(':')[0])
      new_offset = float(output_note[1].split(':')[1])
      output_pitch = temp
      # pattern is a chord
      if ('.' in output_pitch) or output_pitch.isdigit():
          notes_in_chord = output_pitch.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.duration.quarterLength = float(duration)
          new_chord.offset = offset
          output_notes.append(new_chord)
      else:
          new_note = note.Note(output_pitch)
          new_note.offset = offset
          new_note.duration.quarterLength = float(duration)
          output_notes.append(new_note)
      offset += new_offset

  midi_stream = stream.Stream(output_notes)
  piano = instrument.Piano()
  midi_stream.insert(0, piano)
  midi_stream.write('midi', fp='/content/drive/My Drive/MRP/test_file_' + str(version_num) + '_' + str(file_num) + '.midi')

  return

def generate(version_num, file_num, midi_input=False, load=False, num_notes=500):
  seq_len = 60

  if midi_input:
    with open('simple_notes_offsets_midi.pkl', 'rb') as f:
      midi_note = pickle.load(f)

    midi_note = flatten(midi_note)

  with open('simple_notes_offsets.pkl', 'rb') as f:
    notes = pickle.load(f)

  notes = flatten(notes)
  note_set_len = len(set(notes))

  if midi_input:
    note_input = sequence(midi_note, notes, note_set_len, seq_len, midi_input)
  else:
    note_input = sequence(notes, notes, note_set_len, seq_len, midi_input)

  model = create_model(note_set_len, version_num)
  generated_notes = note_prediction(model, note_input, notes, seq_len, num_notes=num_notes, midi_input=midi_input)

  generate_midi(generated_notes, file_num, version_num)

  return




In [ ]:
for x in range(1,5):
  generate('v5_3', x)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
for x in range (1,6):
  generate("v5_3", "Mozart_2" + "_" + str(x), midi_input=True, num_notes = 200)

<ipython-input-6-7146120da1ed>:83: RuntimeWarning: divide by zero encountered in log
  predicted_note = np.log(predicted_note) / temp
